In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from datetime import datetime
mpl.use('TkAgg')

In [2]:
list_name = "Carbon-loc-isoth-N2.xlsx"
#list_name = "Silica-loc-isoth1.xlsx"
dataframe_sorb = pd.read_excel(list_name, header=None, sheet_name="Adsorption")
dataframe_desorb = pd.read_excel(list_name, header=None, sheet_name="Desorption")
P_START = 21  # индекс минимального давления
p_array = np.array(dataframe_sorb[0][P_START:])  # сетка давлений
a_array = np.array(dataframe_sorb.iloc[0][1:])  # сетка размеров пор

array_of_isotherms_s = np.empty((len(a_array), len(p_array)))  # массив изотерм с i-м размером пор
for i in range(len(a_array)):  # заполняем массив изотерм из таблицы pandas
    array_of_isotherms_s[i] = dataframe_sorb[i+1][P_START:]
    
array_of_isotherms_d = np.empty((len(a_array), len(p_array)))  # массив изотерм с i-м размером пор
for i in range(len(a_array)):  # заполняем массив изотерм из таблицы pandas
    array_of_isotherms_d[i] = dataframe_desorb[i+1][P_START:]

In [3]:
fig, ax=plt.subplots()  # строим график j-й изотермы из исходного набора
j = 60
ax.plot(p_array, array_of_isotherms_s[j], marker=".", label=f"kernel isotherm sorption №{j}, pore size = {a_array[j]} nm")
ax.plot(p_array, array_of_isotherms_d[j], marker=".", label=f"kernel isotherm desorption №{j}, pore size = {a_array[j]} nm")

# j1 = j+1
# ax.plot(p_array, array_of_isotherms_s[j1], marker=".", label=f"kernel isotherm sorption №{j1}, pore size = {a_array[j1]} nm")
# ax.plot(p_array, array_of_isotherms_d[j1], marker=".", label=f"kernel isotherm desorption №{j1}, pore size = {a_array[j1]} nm")

ax.set_ylabel("sorption")
ax.set_xlabel("pressure")
legend = ax.legend(loc='upper right')
plt.show() 

In [4]:
with open("data/kernel_generated/Kernel_Carbon-loc-isoth-N2.xlsx_Adsorption.npy", 'rb') as f:
    new_a_array_s = np.load(f)

with open("data/kernel_generated/Kernel_Carbon-loc-isoth-N2.xlsx_Desorption.npy", 'rb') as f:
    new_a_array_d = np.load(f)
    
with open("data/kernel_generated/Size_Carbon-loc-isoth-N2.xlsx_Adsorption.npy", 'rb') as f:
    a_array_s = np.load(f)

with open("data/kernel_generated/Size_Carbon-loc-isoth-N2.xlsx_Desorption.npy", 'rb') as f:
    a_array_d = np.load(f)

In [5]:
fig, ax=plt.subplots()
j = 220

j1  = sorted((np.abs(a_array_d - a_array_s[j])).argsort()[:1])[0]  # находим индексы ближайших к текущему размеров пор


ax.plot(p_array, new_a_array_s[j], marker=".", label=f"kernel isotherm sorption №{j}, pore size = {a_array_s[j]} nm")
ax.plot(p_array, new_a_array_d[j1], marker=".", label=f"kernel isotherm desorption №{j1}, pore size = {a_array_d[j1]} nm")

ax.set_ylabel("sorption")
ax.set_xlabel("pressure")
legend = ax.legend(loc='upper right')
plt.show() 

In [142]:
#  обработка изотерм со скачком
pore_size_jump_pressure_d = pd.DataFrame(columns=['pore_size', 'pressure_of_jump'])  # таблица размер пор - давление при котором происходит скачок в изотерме с данным размером пор
for i, sorption_a in enumerate(array_of_isotherms_d):
    sorption_delta_array = sorption_a[1:] - sorption_a[:-1]
    jump_i = np.argmax(sorption_delta_array)  # индекс скачка 
    jump_pressure = p_array[jump_i]  #  давление скачка
    pore_size_jump_pressure_d.loc[len(pore_size_jump_pressure_d.index)] = [a_array[i], jump_pressure]
    
pore_size_jump_pressure_s = pd.DataFrame(columns=['pore_size', 'pressure_of_jump'])  # таблица размер пор - давление при котором происходит скачок в изотерме с данным размером пор
for i, sorption_a in enumerate(array_of_isotherms_s):
    sorption_delta_array = sorption_a[1:] - sorption_a[:-1]
    jump_i = np.argmax(sorption_delta_array)  # индекс скачка 
    jump_pressure = p_array[jump_i]  #  давление скачка
    pore_size_jump_pressure_s.loc[len(pore_size_jump_pressure_s.index)] = [a_array[i], jump_pressure]
    
pore_size_jump_pressure_s[90:100], pore_size_jump_pressure_d[90:100]

(    pore_size  pressure_of_jump
 90  11.443000             0.854
 91  11.843599             0.862
 92  12.258101             0.868
 93  12.687100             0.874
 94  13.131200             0.878
 95  13.590800             0.884
 96  14.066401             0.890
 97  14.558800             0.894
 98  15.068300             0.900
 99  15.595700             0.904,
     pore_size  pressure_of_jump
 90  11.443000             0.786
 91  11.843599             0.794
 92  12.258101             0.802
 93  12.687100             0.808
 94  13.131200             0.816
 95  13.590800             0.824
 96  14.066401             0.830
 97  14.558800             0.836
 98  15.068300             0.842
 99  15.595700             0.848)

In [7]:
# Строим изотермы десорбции с размерами пор, взятыми из уже сгенерированных изотерм сорбции
def get_k_b(x1, y1, x2, y2):
    k = (y1 - y2) / (x1 - x2)
    b = y1 - k*x1
    return k, b

A_START = 21
def generate_kernel_isotherm(step_i):
    current_a_size = a_array_s[step_i]
    new_isotherm = np.zeros(shape=p_array.shape)
    j1, j2  = sorted((np.abs(a_array - current_a_size)).argsort()[:2])  # находим индексы ближайших к текущему размеров пор
    
    if current_a_size == a_array[j1]:
        return array_of_isotherms_d[j1]
    if current_a_size == a_array[j2]:
        return array_of_isotherms_d[j2]
    
    i = 0
    while p_array[i] <= pore_size_jump_pressure_d["pressure_of_jump"][j1]:
        new_isotherm[i] = np.interp(current_a_size, xp=(a_array[j1], a_array[j2]), fp=(array_of_isotherms_d[j1][i], array_of_isotherms_d[j2][i]))
        i += 1
    tmp = i
    
    i = len(p_array) - 1
    while p_array[i] > pore_size_jump_pressure_d["pressure_of_jump"][j2]:
        new_isotherm[i] = np.interp(current_a_size, xp=(a_array[j1], a_array[j2]), fp=(array_of_isotherms_d[j1][i], array_of_isotherms_d[j2][i]))
        i -= 1
    
    step_pressure = np.interp(current_a_size, (pore_size_jump_pressure_d["pore_size"][j1], pore_size_jump_pressure_d["pore_size"][j2]),
                              (pore_size_jump_pressure_d["pressure_of_jump"][j1], pore_size_jump_pressure_d["pressure_of_jump"][j2]))
    
    while p_array[i] >= step_pressure:
        k, b = get_k_b(p_array[i+2], new_isotherm[i+2],  p_array[i+1],  new_isotherm[i+1])
        new_isotherm[i] =  p_array[i] * k + b
        i -= 1
    
    i = tmp
    while p_array[i] <= step_pressure:
        k, b = get_k_b(p_array[i-2], new_isotherm[i-2],  p_array[i-1],  new_isotherm[i-1])
        new_isotherm[i] =  p_array[i] * k + b
        i += 1
    
    return new_isotherm
    
    
result = []
for step_i in range(len(new_a_array_s)):
     result.append(generate_kernel_isotherm(step_i))

In [18]:
fig, ax=plt.subplots()
j = 304
ax.plot(p_array, new_a_array_s[j], marker="*", label=f"new sorption №{j}, pore size = {a_array_s[j]} nm")
ax.plot(p_array, result[j], marker=".", label=f"new desorption №{j1}, pore size = {a_array_s[j]} nm")

j1, j2  = sorted((np.abs(a_array - a_array_s[j])).argsort()[:2])
ax.plot(p_array, array_of_isotherms_s[j1], marker="*", label=f"kernel sorption №{j1}, pore size = {a_array[j1]} nm")
ax.plot(p_array, array_of_isotherms_d[j1], marker=".", label=f"kernel desorption №{j1}, pore size = {a_array[j1]} nm")
ax.plot(p_array, array_of_isotherms_s[j2], marker="*", label=f"kernel sorption №{j2}, pore size = {a_array[j2]} nm")
ax.plot(p_array, array_of_isotherms_d[j2], marker=".", label=f"kernel desorption №{j2}, pore size = {a_array[j2]} nm")

legend = ax.legend(loc='upper right')
plt.show() 

In [245]:
with open(f'data/kernel_generated/Kernel_test_desorb.npy', 'wb') as f:
    np.save(f, np.array(result))


In [148]:
# Пробуем другой подход для десорбционных изотерм, делаем отдельную более подробную сетку давлений

part_of_new_p_array = np.interp(a_array_s, a_array, pore_size_jump_pressure_d["pressure_of_jump"].to_numpy())[40:]
i  = (np.abs(p_array - part_of_new_p_array[-1])).argsort()[:1][0]

new_p_array = np.concatenate((p_array[:40], part_of_new_p_array, p_array[i+1:]), axis=0)
fig, ax = plt.subplots()
ax.plot(p_array, marker=".")
ax.plot(new_p_array, marker=".")
plt.show() 

In [149]:
# Строим изотермы десорбции с размерами пор, взятыми из уже сгенерированных изотерм сорбции
def generate_kernel_isotherm(step_i):
    current_a_size = a_array_s[step_i]
    new_isotherm = np.zeros(shape=new_p_array.shape)
    j1, j2  = sorted((np.abs(a_array - current_a_size)).argsort()[:2])  # находим индексы ближайших к текущему размеров пор
    
    if current_a_size == a_array[j1]:
        return np.interp(new_p_array, p_array, array_of_isotherms_d[j1])
    if current_a_size == a_array[j2]:
        return np.interp(new_p_array, p_array, array_of_isotherms_d[j2])
    
    i = 0
    while new_p_array[i] <= pore_size_jump_pressure_d["pressure_of_jump"][j1]:
        i1, i2  = sorted((np.abs(p_array - new_p_array[i])).argsort()[:2])
        s1 = np.interp(new_p_array[i], (p_array[i1], p_array[i2]),(array_of_isotherms_d[j1][i1], array_of_isotherms_d[j1][i2]))
        s2 = np.interp(new_p_array[i], (p_array[i1], p_array[i2]),(array_of_isotherms_d[j2][i1], array_of_isotherms_d[j2][i2]))
        new_isotherm[i] = np.interp(current_a_size, xp=(a_array[j1], a_array[j2]), fp=(s1, s2))
        i += 1
    tmp = i
    
    i = len(new_p_array) - 1
    while p_array[min((np.abs(p_array - new_p_array[i])).argsort()[:2])] > pore_size_jump_pressure_d["pressure_of_jump"][j2]:
        i1, i2  = sorted((np.abs(p_array - new_p_array[i])).argsort()[:2])
        s1 = np.interp(new_p_array[i], (p_array[i1], p_array[i2]),(array_of_isotherms_d[j1][i1], array_of_isotherms_d[j1][i2]))
        s2 = np.interp(new_p_array[i], (p_array[i1], p_array[i2]),(array_of_isotherms_d[j2][i1], array_of_isotherms_d[j2][i2]))
        new_isotherm[i] = np.interp(current_a_size, xp=(a_array[j1], a_array[j2]), fp=(s1, s2))
        i -= 1
    
    step_pressure = new_p_array[step_i]
    
    while new_p_array[i] >= step_pressure:
        k, b = get_k_b(new_p_array[i+2], new_isotherm[i+2],  new_p_array[i+1],  new_isotherm[i+1])
        new_isotherm[i] =  new_p_array[i] * k + b
        i -= 1

    i = tmp
    while new_p_array[i] <= step_pressure:
        k, b = get_k_b(new_p_array[i-2], new_isotherm[i-2],  new_p_array[i-1],  new_isotherm[i-1])
        new_isotherm[i] =  new_p_array[i] * k + b
        i += 1

    return new_isotherm
    
    
result = []
for step_i in range(len(new_a_array_s)):
     result.append(generate_kernel_isotherm(step_i))

In [150]:
fig, ax=plt.subplots()
j = 305
ax.plot(p_array, new_a_array_s[j], marker="*", label=f"new sorption №{j}, pore size = {a_array_s[j]} nm")
ax.plot(new_p_array, result[j], marker=".", label=f"new desorption №{j1}, pore size = {a_array_s[j]} nm")

j1, j2  = sorted((np.abs(a_array - a_array_s[j])).argsort()[:2])
ax.plot(p_array, array_of_isotherms_s[j1], marker="*", label=f"kernel sorption №{j1}, pore size = {a_array[j1]} nm")
ax.plot(p_array, array_of_isotherms_d[j1], marker=".", label=f"kernel desorption №{j1}, pore size = {a_array[j1]} nm")
ax.plot(p_array, array_of_isotherms_s[j2], marker="*", label=f"kernel sorption №{j2}, pore size = {a_array[j2]} nm")
ax.plot(p_array, array_of_isotherms_d[j2], marker=".", label=f"kernel desorption №{j2}, pore size = {a_array[j2]} nm")

legend = ax.legend(loc='upper right')
plt.show() 

In [153]:
with open(f'data/kernel_generated/Kernel_test2_desorb.npy', 'wb') as f:
    np.save(f, np.array(result))
with open(f'data/kernel_generated/pressure_test2_desorb.npy', 'wb') as f:
    np.save(f, np.array(new_p_array))